In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

#automatically detect common phrases (bigrams) from a list of sentences.
from gensim.models.phrases import Phrases, Phraser

from gensim.models import Word2Vec

In [3]:
df = pd.read_csv('../input/rickmorty-scripts/RickAndMortyScripts.csv')
df.shape

(1905, 6)

In [4]:
df.head()

,index,season no.,episode no.,episode name,name,line
0,0,1,1,Pilot,Rick,Morty! You gotta come on. Jus'... you gotta co...
1,1,1,1,Pilot,Morty,"What, Rick? What’s going on?"
2,2,1,1,Pilot,Rick,"I got a surprise for you, Morty."
3,3,1,1,Pilot,Morty,It's the middle of the night. What are you tal...
4,4,1,1,Pilot,Rick,"Come on, I got a surprise for you. Come on, h..."


In [5]:
df.isnull().sum()

index           0
season no.      0
episode no.     0
episode name    0
name            0
line            0
dtype: int64

In [7]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['line'])

In [9]:
df_clean = pd.DataFrame({'clean': brief_cleaning})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(1832, 1)

In [10]:
sent = [row.split() for row in df_clean['clean']]

In [11]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

In [15]:
bigram = Phraser(phrases)
print(bigram)

FrozenPhrases<1 phrases, min_count=30, threshold=10.0>


In [17]:
pre_sentences = bigram[sent]

In [30]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     vector_size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                    )

In [31]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.0 mins


In [32]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 0.03 mins


In [34]:
w2v_model.wv.most_similar(positive=["rick"])

[('her', 0.999716579914093),
 ('when', 0.9997146129608154),
 ('hey', 0.9997047185897827),
 ('for', 0.9997045993804932),
 ('little', 0.9997045397758484),
 ('things', 0.9997042417526245),
 ('want', 0.9997031688690186),
 ('vindicators', 0.999701976776123),
 ('were', 0.9997014999389648),
 ('morty', 0.9997007250785828)]

In [38]:
w2v_model.wv.most_similar('morty')


[('bad', 0.9997394680976868),
 ('were', 0.9997340440750122),
 ('things', 0.9997249245643616),
 ("can't", 0.9997173547744751),
 ('with', 0.9997159242630005),
 ('then', 0.9997128248214722),
 ('hey', 0.9997127652168274),
 ('re', 0.9997114539146423),
 ('about', 0.9997111558914185),
 ('maybe', 0.9997110366821289)]

In [40]:
w2v_model.wv.save_word2vec_format('output.txt')